# 👕 코디 크롤링 노트북

## 크롤링 목록
- 스타일
- 이미지 소스
- 코디 고유 ID
- 태그
- 코디에 포함된 item들의 ID

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from easydict import EasyDict
from tqdm import tqdm

# pip install openpyxl
import openpyxl
import time

In [2]:
# 코디들을 뽑아올 페이지 지정
COORDI_LIST_PATH = 'https://www.musinsa.com/app/styles/lists'
COORDI_BASE_PATH = "https://www.musinsa.com/app/styles/views/"

In [3]:
# 🚀 Chrome option 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(argument='--headless')
chrome_options.add_argument(argument='--no-sandbox')
chrome_options.add_argument(argument='--disable-dev-shm-usage')

In [4]:
# 🚀 크롤링 완료된 정보를 저장할 excel sheet_codi 지정
wb_codi = openpyxl.Workbook()
sheet_codi = wb_codi.active
sheet_codi.append(["id",  "style", "img_url", "url"])

wb_codi_tag = openpyxl.Workbook()
sheet_codi_tag = wb_codi_tag.active
sheet_codi_tag.append(["id", "tag"])

wb_codi_item_id = openpyxl.Workbook()
sheet_codi_item_id = wb_codi_item_id.active
sheet_codi_item_id.append(["id", "item_id"])

In [5]:
# 🚀 최상위 메인 페이지 불러오기 (코디 목록 60개 보여지는 페이지)
driver = webdriver.Chrome(executable_path='chromedriver', options=chrome_options)
driver.get(COORDI_LIST_PATH)
time.sleep(3)

# 🚀 (코디 링크, 스타일 정보, 코디 이미지 url) 리스트 받아오기
codi_link_result = driver.find_elements(by=By.CSS_SELECTOR, value=".style-list-item__thumbnail > a")
driver.implicitly_wait(3)
style_result = driver.find_elements(by=By.CSS_SELECTOR, value=".style-list-information__text")
driver.implicitly_wait(3)
image_url_result = driver.find_elements(by=By.CSS_SELECTOR, value=".style-list-item__thumbnail > a > div.style-list-thumbnail > img")
driver.implicitly_wait(3)

print ("현재 페이지에 존재하는 코디의 수 확인 (3개 숫자 동일)")
print (len(codi_link_result), len(style_result), len(image_url_result))

codi_id_list = list()
codi_style_list = list()
codi_url_list = list()

# ⭐ 1. 코디 ID 정보 javascript function 인자로 부터 받아오기
for link_element in codi_link_result:
    js_function = link_element.get_attribute("onclick")
    codi_id = js_function.split("'")[1]
    codi_id_list.append(codi_id)
    
# ⭐ 2. style 정보 text로 추출하기
for style_element in style_result:
    codi_style_list.append(style_element.text)
    
# ⭐ 3. 코디의 이미지 url 받아오기
for img_element in image_url_result:
    codi_url_list.append(img_element.get_attribute('src'))

현재 페이지에 존재하는 코디의 수 확인 (3개 숫자 동일)
60 60 60


In [6]:
# 🚀 코디 링크에 하나씩 접속하면서, 연관된 상품 ID, 코디태그 받아오기
for codi_id, codi_style, codi_img_url in tqdm(zip(codi_id_list, codi_style_list, codi_url_list), total=len(codi_id_list)):
        
    # 🚀 하나의 코디에 대한 information 저장
    codi_info = list()
    
    # ⭐ 4. 코디의 상세정보가 있는 url 받아오기
    codi_path = COORDI_BASE_PATH + codi_id
    
    # 🚀 코디의 경로 받아오고 코디 상세정보 페이지 진입하기
    driver.get(codi_path)
    driver.implicitly_wait(3) #페이지를 로딩하는 시간동안 대기
    
    # ⭐ 5. 코디 태그 받아오기 (다른 sheet에 저장)
    coordi_tags = driver.find_elements(by=By.CSS_SELECTOR, value=".ui-tag-list__item")
    for tag_element in coordi_tags:
        sheet_codi_tag.append([codi_id, tag_element.text])
    
    # ⭐ 6. 현재 코디에 포함된 아이템 id 받아오기 (다른 sheet에 저장)
    item_elements = driver.find_elements(by=By.CSS_SELECTOR, value=".styling_img")
    for item_element in item_elements:
        item_id = item_element.get_attribute("href").split("/")[-2]
        sheet_codi_item_id.append([codi_id, item_id])
        
    codi_info.append(codi_id)
    codi_info.append(codi_style)
    codi_info.append(codi_img_url)
    codi_info.append(codi_path)
    sheet_codi.append(codi_info)

wb_codi.save("./asset/codi.csv")
wb_codi_tag.save("./asset/codi_tag.csv")
wb_codi_item_id.save("./asset/codi_item_id.csv")

driver.close()

100%|██████████| 60/60 [01:33<00:00,  1.56s/it]


FileNotFoundError: [Errno 2] No such file or directory: './asset/codi.csv'